In [1]:
#universal sentence encoder
import pandas as pd
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import numpy as np
import tqdm
            

import spacy
from nltk.corpus import stopwords 
stops = list(stopwords.words('english'))
other_stops = ['author', 'funder', 'all', 'rights', 'reuse', 'allowed', ,'also', 'use', 'without', 'permission', 'the', 'copyright', 'holder', 'for',\
                     'this', 'preprint', 'which', 'was', 'not', 'peer', 'reviewed', 'is', 'the', 'doi', 'biorxiv', 'preprint', 'author',\
                     'funder', 'all', 'reserved', 'copyright', 'holder', 'for', 'this', 'preprint', 'which', 'peer', 'reviewed', 'doi', 'biorxiv', 'preprint',
                'the copyright holder for this preprint (which was not peer-reviewed) is the . doi doi: biorxiv preprint author/funder.', \
                'all rights reserved.', ' no reuse allowed without permission.the copyright holder for this preprint (which was not peer-reviewed)', \
               'biorxiv preprint the copyright holder for this preprint (which was not peer-reviewed) is the . doi doi: biorxiv preprint significance is shown comparing the replication of c11 pk disrupt and c11 pk restore (aii).', \
               ' significance shown is compared to wt replicon. error bars are calculated by sem, n = 3, * p 673 < 0 05, **** p < 0 0001  674 author/funder.',
               'all rights reserved. no reuse allowed without permission.the copyright holder for this preprint (which was not peer-reviewed) is the . doi doi: biorxiv preprint 33 675 author/funder.',\
               ' all rights reserved. no reuse allowed without permission.the copyright holder for this preprint (which was not peer-reviewed) is the . doi doi: biorxiv preprint ']
stops = stops + other_stops


#https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

from pprint import pprint

from IPython.display import Image
import re



unable to import 'smart_open.gcs', disabling that module


# Important
This is an LDA model, therefore we can ignore elements of semantics, syntax, etc. We are utilizing Bag of Words methodologies, and therefore do not need to relate features in terms of contextual information, i.e, each document is a bag of words that can be used to inform a distribution that will be sampled.

Sampling can be done via gibbs iterative sampling, or bayes.


IMPORTANT:
CODE IS SLOW, NEED TO MOVE FROM PANDAS -> DASK

In [2]:
print(type(stops))

<class 'list'>


In [3]:
#refer to json_to_csv.ipynb for creating the csv below
df = pd.read_csv('C:/Users/eovit/Python/Kaggle/research_pdfpapers_v1.csv')

In [4]:
df.head()

,Unnamed: 0,title,authors,abstract,all_body
0,0,Absence of surface expression of feline infect...,"['E Cornelissen', ""H ['L'] Dewerchin"", ""E ['Va...",Feline infectious peritonitis virus (FIPV) pos...,Feline infectious peritonitis (FIP) is a fatal...
1,1,Title: Rethinking high-risk groups in COVID-19,"['Anastasia Vishnevetsky', 'Michael Levy']",NaN,How do we protect our 'high-risk' patient popu...
2,2,Plasma inflammatory cytokines and chemokines i...,"[""W ['K'] Lam"", ""C ['K'] Wong"", ""C ['W K'] Lam...",Severe acute respiratory syndrome (SARS) is a ...,Severe acute respiratory syndrome (SARS) is a ...
3,3,NaN,"[""Chris ['J'] Williams"", 'Brunhilde Schweiger'...",Background: Influenza immunisation for healthc...,The German standing commission for immunisatio...
4,4,Disaster Perceptions,[],NaN,". So, if there is no singular definition of ri..."


In [5]:
#below is common preprocessing, motivated by the code at: https://www.kaggle.com/matteomuffo/a-fine-grained-covid-19-question-answering-engine
def preprocessing(text):
    # remove mail
    text = re.sub(r'[a-z0-9._%+-]+@[a-z0-9.-]+\.[a-z]{2,}', 'MAIL', text)
    # remove doi
    text = re.sub(r'https\:\/\/doi\.org[^\s]+', 'DOI', text)
    # remove https
    text = re.sub(r'(\()?\s?http(s)?\:\/\/[^\)]+(\))?', r'\g<1>LINK\g<3>', text)
    # remove single characters repeated at least 3 times for spacing error (e.g. s u m m a r y)
    text = re.sub(r'(\w\s+){3,}', ' ', text)
    # replace tags (e.g. [3] [4] [5]) with whitespace
    text = re.sub(r'(\[\d+\]\,?\s?){3,}(\.|\,)?', r' \g<2>', text)
    # replace tags (e.g. [3, 4, 5]) with whitespace
    text = re.sub(r'\[[\d\,\s]+\]', ' ',text)
     # replace tags (e.g. (NUM1) repeated at least 3 times with whitespace
    text = re.sub(r'(\(\d+\)\s){3,}', ' ',text)
    # remove all full stops as abbreviations (e.g. i.e. cit. and so on)
    text = re.sub(r'\.(\s)?([^A-Z\s])', r' \g<1>\g<2>', text)
    # return lowercase text
    return text.lower()

#secondary preprocessing incase we miss anything
#source https://tedboy.github.io/nlps/generated/generated/gensim.utils.simple_preprocess.html
def ppreprocess(data):
 
    temp = data.copy()
    temp = [re.sub(r"\'", "", x) for x in temp] #remove single quotes
    temp = [re.sub(r"\,", "", x) for x in temp] #remove commas, technically don't need to but I like too
    for sentence in temp:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

In [6]:
df.all_body = df.all_body.apply(preprocessing)


In [7]:
print(f'The size of our df is, {df.shape}')

The size of our df is, (59561, 5)


In [8]:
#lets make sure that things appear the way we want them to
df.head() 

,Unnamed: 0,title,authors,abstract,all_body
0,0,Absence of surface expression of feline infect...,"['E Cornelissen', ""H ['L'] Dewerchin"", ""E ['Va...",Feline infectious peritonitis virus (FIPV) pos...,feline infectious peritonitis (fip) is a fatal...
1,1,Title: Rethinking high-risk groups in COVID-19,"['Anastasia Vishnevetsky', 'Michael Levy']",NaN,how do we protect our 'high-risk' patient popu...
2,2,Plasma inflammatory cytokines and chemokines i...,"[""W ['K'] Lam"", ""C ['K'] Wong"", ""C ['W K'] Lam...",Severe acute respiratory syndrome (SARS) is a ...,severe acute respiratory syndrome (sars) is a ...
3,3,NaN,"[""Chris ['J'] Williams"", 'Brunhilde Schweiger'...",Background: Influenza immunisation for healthc...,the german standing commission for immunisatio...
4,4,Disaster Perceptions,[],NaN,". so, if there is no singular definition of ri..."


In [9]:
df.all_body.iloc[0]

"feline infectious peritonitis (fip) is a fatal chronic disease in cats caused by a coronavirus, feline infectious peritonitis virus (fipv), and characterized by granulomatous lesions formed at the serosae of different organs. two forms can be distinguished. cats suffering from the wet or effusive form have exudates in their body cavities. exudate is absent in the second form, hence the name dry or non-effusive form.fipv-infected cells are detected in the pyogranulomas and, based on morphology and the granulocyte/ monocyte/macrophage marker calprotectin, defined as macrophages (weiss and scott, 1981; kipar et al , 1998) . infected mononuclear cells were also isolated from exudates (cammarata parodi et al , 1993; paltrinieri et al , 1999) . in one way or another, these infected cells succeed in staying alive and transmitting www elsevier com/locate/vetmic veterinary microbiology 121 (2007)  virus to new susceptible cells in the presence of a high concentration of antibodies. how they do

In [10]:
#create a collection of all the elements in the all_body column
data = df.all_body.values.tolist()

In [11]:
#apply secondary preprocessing (which yields a generator) and format it as a list into a manageable data structure 
#THIS IS A LIST OF LISTS REMEMBER THIS
corpus = list(ppreprocess(data))

In [12]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(corpus, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[corpus], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
#interesting to note that we do not see any trigrams, indicating that we did not meet the threshold, which we could perhaps lower
print(trigram_mod[bigram_mod[corpus[0]]]);

['feline', 'infectious_peritonitis_fip', 'is', 'fatal', 'chronic', 'disease', 'in', 'cats', 'caused', 'by', 'coronavirus', 'feline', 'infectious', 'peritonitis', 'virus', 'fipv', 'and', 'characterized', 'by', 'granulomatous', 'lesions', 'formed', 'at', 'the', 'serosae', 'of', 'different', 'organs', 'two', 'forms', 'can', 'be', 'distinguished', 'cats', 'suffering', 'from', 'the', 'wet', 'or', 'effusive_form', 'have', 'exudates', 'in', 'their', 'body', 'cavities', 'exudate', 'is', 'absent', 'in', 'the', 'second', 'form', 'hence', 'the', 'name', 'dry', 'or', 'non', 'effusive_form', 'fipv', 'infected', 'cells', 'are', 'detected', 'in', 'the', 'pyogranulomas', 'and', 'based', 'on', 'morphology', 'and', 'the', 'granulocyte', 'monocyte_macrophage', 'marker', 'calprotectin', 'defined', 'as', 'macrophages', 'weiss', 'and', 'scott_kipar', 'et', 'al', 'infected', 'mononuclear', 'cells', 'were', 'also', 'isolated', 'from', 'exudates', 'cammarata_parodi', 'et', 'al', 'paltrinieri', 'et', 'al', 'in'

In [13]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    """
    Input: texts <collection of documents, aka corpus>
    ------------
    Output: return tokenized text that meets return criteria <gensim.utils.simple_preprocessing> and LEN(WORD) > 3
    """
    #remove words less than 3 can remove certain redundancies in the paper
    return [[word for word in gensim.utils.simple_preprocess(str(doc)) if word not in stops and len(word) > 3] for doc in texts]

def make_bigrams(texts):
    """
    Input: texts <collection of documents, aka corpus>
    ------------
    Output: tokenized bigram representation of each document
    """
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    """
    Input: texts <collection of documents, aka corpus>
    ------------
    Output: tokenized trigram representation of each document
    """
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """
    Input: texts <collection of documents, aka corpus>
           list of POSTAGS check: https://spacy.io/api/annotation
    ------------
    Output: leammatized representation of corpus, with accompanying tagging from Spacy
    """
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out


In [14]:
# Remove Stop Words
corpus_nostops = remove_stopwords(corpus)

# Form Bigrams
corpus_bigrams = make_bigrams(corpus_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en

#this part can be made much more efficient


nlp = spacy.load('en', disable=['parser', 'ner'])
nlp.max_length =  2000000
# Do lemmatization keeping only noun, adj, vb, adv
corpus_lemmatized = lemmatization(corpus_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(corpus_lemmatized[:1]);

[['feline', 'infectious', 'peritonitis', 'fatal', 'chronic', 'disease', 'cat', 'cause', 'infectious', 'peritonitis', 'virus', 'fipv', 'characterize', 'granulomatous', 'lesion', 'form', 'serosae', 'different', 'organ', 'form', 'distinguish', 'cat', 'suffer', 'effusive', 'form', 'exudate', 'body', 'cavity', 'exudate', 'absent', 'second', 'form', 'hence', 'name', 'effusive', 'form', 'fipv', 'infect', 'cell', 'detect', 'base', 'morphology', 'granulocyte', 'monocyte_macrophage', 'marker', 'calprotectin', 'define', 'infect', 'mononuclear', 'cell', 'also', 'isolate', 'exudate', 'infected', 'cell', 'succeed', 'stay', 'alive', 'transmitting', 'elsevi', 'locate_vetmic', 'veterinary', 'microbiology', 'virus', 'susceptible', 'cell', 'presence', 'high', 'concentration', 'antibody', 'know', 'infected', 'cell', 'normally', 'eliminate', 'adaptive', 'immune', 'system', 'antibody', 'mediate', 'lysis', 'cell', 'mediate', 'lysis', 'antibody', 'mediate', 'lysis', 'presence', 'antigen', 'surface', 'infected

In [15]:
#this part takes forever
#lets put the clean stuff into a csv
#instantiate a empty data frame so we can add the clean data to it

#Does not work yet


# temp = pd.DataFrame()

In [16]:
# temp['cleaned_text'] = pd.Series(corpus_lemmatized)

In [17]:
# temp.to_csv('cleaned_text_LDA_520.csv') #size of this output is 1.2GB

In [18]:
#EVERYTHING BEFORE CREATED THIS TEXT 
# corpus_lemmatized = pd.read_csv('cleaned_text_LDA_520.csv')

In [19]:
# corpus_lemmatized[0].replace('"','')

In [20]:
corpus_lemmatized[2]

['severe',
 'recently',
 'emerge',
 'infectious',
 'disease',
 'characterize',
 'persistent',
 'fever',
 'respiratory',
 'symptom',
 'lung',
 'consolidation',
 'respiratory',
 'failure',
 'case',
 'also',
 'manifest',
 'case',
 'find',
 'lymphopenia',
 'depletion',
 'lymphocyte',
 'could',
 'associate',
 'related',
 'death',
 'result',
 'mainly',
 'pulmonary',
 'complication',
 'include',
 'progressive',
 'respiratory',
 'failure',
 'alveolar',
 'damage',
 'acute',
 'ard',
 'identify',
 'genome',
 'sequence',
 'moderately',
 'relate',
 'known',
 'coronaviruse',
 'rapid',
 'diagnostic',
 'test',
 'use',
 'molecular',
 'technique',
 'detection',
 'develop',
 'recently',
 'administration',
 'antiviral',
 'drug',
 'ribarivin',
 'together',
 'systemic',
 'result',
 'alleviation',
 'inflammation',
 'reduction',
 'improvement',
 'survival',
 'patient',
 'control',
 'clinical',
 'trial',
 'require',
 'confirm',
 'regimen',
 'genuine',
 'efficacy',
 'show',
 'production',
 'specific',
 'inflamm

In [21]:
#lets create the main inputs into the LDA model, which are the id2word(doc), which is dictionary of the word index adn frequency per document
#the other input to add is the corpus , which is the entire id2word(doc) collection, so it can be thought of as a nested dictionary 
# Create Dictionary
id2word = corpora.Dictionary(corpus_lemmatized)

# Create Corpus
texts = corpus_lemmatized

# Term Document Frequency
f_corpus = [id2word.doc2bow(text) for text in texts]

# View
print(f_corpus[:1]);

[[(0, 3), (1, 4), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 2), (8, 2), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 5), (17, 1), (18, 4), (19, 2), (20, 25), (21, 27), (22, 1), (23, 2), (24, 1), (25, 2), (26, 3), (27, 4), (28, 1), (29, 2), (30, 2), (31, 1), (32, 1), (33, 1), (34, 1), (35, 3), (36, 15), (37, 3), (38, 1), (39, 72), (40, 3), (41, 1), (42, 1), (43, 1), (44, 2), (45, 1), (46, 3), (47, 1), (48, 1), (49, 3), (50, 2), (51, 3), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 2), (61, 1), (62, 1), (63, 2), (64, 2), (65, 1), (66, 1), (67, 1), (68, 2), (69, 1), (70, 1), (71, 1), (72, 1), (73, 2), (74, 2), (75, 1), (76, 2), (77, 1), (78, 3), (79, 1), (80, 5), (81, 1), (82, 1), (83, 4), (84, 1), (85, 3), (86, 2), (87, 3), (88, 1), (89, 2), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 4), (96, 3), (97, 2), (98, 1), (99, 1), (100, 1), (101, 1), (102, 1), (103, 1), (104, 1), (105, 1), (106, 13), (107, 9), (108, 1), (109, 12), (1

In [22]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in f_corpus[:1]]

[[('able', 3),
  ('absence', 4),
  ('absent', 1),
  ('abundantly', 1),
  ('accept', 1),
  ('accessible', 1),
  ('accord', 1),
  ('account', 2),
  ('adaptive', 2),
  ('addition', 1),
  ('affect', 1),
  ('afterwards', 1),
  ('alive', 1),
  ('allow', 1),
  ('almost', 1),
  ('already', 1),
  ('also', 5),
  ('analysis', 1),
  ('analyze', 4),
  ('anti', 2),
  ('antibody', 25),
  ('antigen', 27),
  ('apoptosis', 1),
  ('base', 2),
  ('become', 1),
  ('belong', 2),
  ('bind', 3),
  ('biotinylate', 4),
  ('black', 1),
  ('block', 2),
  ('blood', 2),
  ('body', 1),
  ('buckinghamshire', 1),
  ('calprotectin', 1),
  ('cap', 1),
  ('capable', 3),
  ('cat', 15),
  ('cause', 3),
  ('cavity', 1),
  ('cell', 72),
  ('cellular', 3),
  ('centrifugation', 1),
  ('centrifuge', 1),
  ('characterize', 1),
  ('chronic', 2),
  ('classical', 1),
  ('clear', 3),
  ('clinical', 1),
  ('clinician', 1),
  ('collect', 3),
  ('complement', 2),
  ('complex', 3),
  ('complicated', 1),
  ('concentration', 1),
  ('concl

In [ ]:
Image(filename = "C:/Users/eovit/Python/Kaggle/Lda.png")
#where corpus is collectino of M documents [D]
#Doc containts N words
#Z is topic selected
#solve p(wN | Zn , ThetaN)

In [23]:
# Build LDA model; can be fine tuned for coherence and perplexity, where K = num_topics, we are updating every iteration

#k is important because it
lda_model = gensim.models.ldamulticore.LdaMulticore(corpus=f_corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           chunksize=100,
                                           workers=11, # Num. Processing Cores - 1         
                                           passes=50,
                                           eval_every= 1,
                                           #important to note that we lose access to auto-configured alpha when using multicore
                                           per_word_topics=True)

In [24]:

pprint(lda_model.print_topics())
doc_lda = lda_model[f_corpus]

[(0,
  '0.021*"animal" + 0.014*"disease" + 0.013*"infection" + 0.012*"virus" + '
  '0.012*"human" + 0.011*"specie" + 0.009*"dog" + 0.009*"cat" + 0.008*"calf" + '
  '0.007*"also"'),
 (1,
  '0.018*"patient" + 0.012*"case" + 0.008*"lesion" + 0.007*"result" + '
  '0.006*"image" + 0.006*"diagnosis" + 0.006*"present" + 0.005*"clinical" + '
  '0.005*"lung" + 0.005*"disease"'),
 (2,
  '0.014*"use" + 0.009*"method" + 0.009*"particle" + 0.008*"system" + '
  '0.008*"high" + 0.007*"surface" + 0.007*"water" + 0.006*"detection" + '
  '0.006*"concentration" + 0.006*"time"'),
 (3,
  '0.052*"cell" + 0.051*"virus" + 0.032*"viral" + 0.027*"infection" + '
  '0.023*"protein" + 0.015*"replication" + 0.013*"infect" + 0.011*"host" + '
  '0.008*"also" + 0.008*"expression"'),
 (4,
  '0.054*"cell" + 0.015*"expression" + 0.012*"mouse" + 0.012*"response" + '
  '0.010*"induce" + 0.009*"increase" + 0.008*"also" + 0.008*"immune" + '
  '0.008*"level" + 0.008*"receptor"'),
 (5,
  '0.033*"sequence" + 0.027*"gene" + 0.02

In [25]:
# Compute Perplexity; lower the better
#these reSULTS ARE BETTER THAN ABSTRACT AND TITLE
#mallet is better than the LDA implementation 
#more data has given us better results (about twice as much informatoin)
print('\nPerplexity: ', lda_model.log_perplexity(f_corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score, interpreted similarly to acc
coherence_model_lda = CoherenceModel(model=lda_model, texts= corpus_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.102888210453523

Coherence Score:  0.5009330792729505


In [26]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, f_corpus, id2word)
vis

c:\users\eovit\miniconda3\envs\virt\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
13    -0.062479  0.103682       1        1  9.274211
4     -0.053156 -0.140594       2        1  8.359242
16    -0.074334  0.122623       3        1  7.344107
17    -0.047895 -0.099864       4        1  6.789949
11    -0.056299  0.098327       5        1  6.643540
12    -0.054823 -0.181203       6        1  6.505297
10    -0.084780  0.076992       7        1  5.721804
3     -0.038070 -0.213142       8        1  5.685942
14    -0.054478  0.113667       9        1  5.541877
5     -0.069505 -0.117777      10        1  5.512819
18    -0.075973  0.011224      11        1  5.314346
1     -0.037535  0.098432      12        1  5.131298
0     -0.045208  0.019382      13        1  4.653657
2     -0.064303 -0.009419      14        1  4.517859
9     -0.057701 -0.111021      15        1  3.940325
19    -0.031790  0.211166      16        1  3.894728
6     -0.038522 -0.056576      17        1  2.854717
8      0.027501  0.082177      18        1  1.950686
7      0.416460 -0.001219      19        1  0.340412
15     0.502891 -0.006859      20        1  0.023183, topic_info=      Category          Freq       Term         Total  loglift  logprob
39     Default  1.288294e+06       cell  1.288294e+06  30.0000  30.0000
449    Default  9.766250e+05    patient  9.766250e+05  29.0000  29.0000
323    Default  8.592010e+05      virus  8.592010e+05  28.0000  28.0000
247    Default  6.635450e+05    protein  6.635450e+05  27.0000  27.0000
147    Default  7.162860e+05  infection  7.162860e+05  26.0000  26.0000
...        ...           ...        ...           ...      ...      ...
11114  Topic20  1.097336e+02  hydrolase  1.165020e+03   6.0071  -5.5020
10362  Topic20  8.433952e+01       mono  1.125327e+03   5.7785  -5.7652
2715   Topic20  4.664715e+01     repair  1.129115e+04   2.8804  -6.3575
611    Topic20  5.422568e+01     damage  4.728828e+04   1.5987  -6.2069
7918   Topic20  3.347321e+01    polymer  8.535831e+03   2.8282  -6.6893

[1809 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
1903      2  0.012960  abdominal
1903      9  0.059742  abdominal
1903     12  0.861354  abdominal
1903     13  0.014751  abdominal
1903     15  0.003161  abdominal
...     ...       ...        ...
2900      7  0.041294   zoonotic
2900      8  0.013183   zoonotic
2900     10  0.083704   zoonotic
2900     13  0.792954   zoonotic
2900     15  0.024274   zoonotic

[8056 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[14, 5, 17, 18, 12, 13, 11, 4, 15, 6, 19, 2, 1, 3, 10, 20, 7, 9, 8, 16])

In [28]:
def compute_coherence_values(corpus, dictionary, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics
    
    THIS DOES NOT TAKE INTO ACCOUNT ALTERATIONS TO ANY HYPERPARAMTERS OTHER THAN K

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    tbar = tqdm.tqdm(total = limit//2)
    for num_topics in range(start, limit, step):
        #better results, however this is about ~5 to 7x slower than Multicore
        model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics= num_topics, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=50,
                                           alpha='auto',
                                           per_word_topics=True)
            
#this version is faster, but produces worst results because we lose access to asymmetric alpha 'auto'    
    
#         model = gensim.models.ldamulticore.LdaMulticore(corpus=corpus,
#                                            id2word=id2word,
#                                            num_topics=20, 
#                                            random_state=100,
#                                            chunksize=100,
#                                            workers=11, # Num. Processing Cores - 1         
#                                            passes=50,
#                                            eval_every= 1,
#                                            #important to note that we lose access to auto-configured alpha when using multicore
#                             

# per_word_topics=True)
        
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
        tbar.update(1)
    
    tbar.close()
    
    return model_list, coherence_values


# supporting function
def fine_tuned_coherence_values(corpus, dictionary, k, a, b):
    """
    Compute c_v coherence for various number of topics, alphas, and eta values
    
    THIS DOES NOT TAKE INTO ACCOUNT ALTERATIONS TO ANY HYPERPARAMTERS OTHER THAN K

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    k: number of topics passed as vector
    a = alpha (auto sets it to 1.0/num_topics)
    b = eta (auto sets: A-priori belief on word probability, can be scalar, vector, etc.)

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA 
    model with respective number of topics
    """
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=10, 
                                           random_state=100,
                                           chunksize=100,
                                           workers=11, # Num. Processing Cores - 1        
                                           passes=50,
                                           alpha=a,
                                           eta=b,
                                           per_word_topics=True)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=dictionary, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [ ]:
model_list, coherence_values = compute_coherence_values(corpus=f_corpus, dictionary=id2word, texts= corpus_lemmatized, start=2, limit=10, step=2)

 20%|██████████▊                                           | 1/5 [1:48:28<7:13:53, 6508.47s/it]

In [ ]:




# Show graph
limit=10; start=2; step=2;

x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()



In [ ]:
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [ ]:

best_model = model_list[4]

best_coh =  coherence_values[4]

print(f'The best overall model = {best_model, best_coh}')


In [ ]:
#is our perplexity better than our manual model?
#-12.14, actually better 
print('\nBest Model Perplexity: ', best_model.log_perplexity(f_corpus))  # a measure of how good the model is. lower the better.

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(best_model, f_corpus, id2word)
vis

Since we have strong overlap between 11 of our topics, we should probably try to play with the hyperparameters to see how this will affect the model topics output.

In [ ]:
pprint(best_model.print_topics())


In [ ]:

grid = {}
grid['Validation_Set'] = {}
# Topics range
min_topics = 4
max_topics = 15
step_size = 1
topics_range = range(min_topics, max_topics, step_size)
# Alpha parameter

alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')
# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')
# Validation sets
num_of_docs = len(corpus)


model_results = {'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }
# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total= 540)
    #we removed the different validation sets:
    # iterate through validation corpuses
    for k in topics_range:
        # iterate through alpha values
        for a in alpha:
             # iterare through beta values
            for b in beta:
                # get the coherence score for the given parameters
                cv = fine_tuned_coherence_values(corpus=f_corpus, dictionary=id2word, \
                                                  k=k, a=a, b=b)
                # Save the model results
                   
                model_results['Topics'].append(k)
                model_results['Alpha'].append(a)
                model_results['Beta'].append(b)
                model_results['Coherence'].append(cv)
                    
                pbar.update(1)
    pd.DataFrame(model_results).to_csv('lda_tuning_results2.csv', index=False)
    pbar.close()


# Assigning topics to each observation
The next step is to use the full list of papers in order to assign each paper entry (that includes the body of the paper).

This can also be done using the abstracts, as the abstract should determine some amount of information, where I = len(abstract)/len(full_paper).


In [ ]:
#inspiration https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/topic_methods.ipynb

#we are using the best model from our coherence test
topics = best_model.get_document_topics(f_corpus, per_word_topics=True) 

all_topics = [(doc_topics, word_topics, word_phis) for doc_topics, word_topics, word_phis in topics]

In [ ]:
all_doc_tops = []
for i in range(len(all_topics)):
    all_doc_tops.append(all_topics[i][0])

In [ ]:
#test this out for arbitrary observation in the corpus

doc_topics, word_topics, phi_values = all_topics[2]
print('Document topic:', doc_topics, "\n")
print('Word topic:', word_topics, "\n")
print('Phi value:', phi_values)

In [ ]:
df['topics_by_doc'] = all_doc_tops

In [ ]:
df

## Next steps
The next step would be to one, test out our LDA by 1) staying current with the CORD19 dataset, tuning our hyperparameters, and lasty, we could extend this code base doing something like a Author Topic model, where we can attribute topics not only by the contents of the document, but the author of the document as well.



\# Bibliography

-SPACY

-GENSIM

-https://towardsdatascience.com/evaluate-topic-model-in-python-latent-dirichlet-allocation-lda-7d57484bb5d0

-https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/topic_methods.ipynb

-https://radimrehurek.com/gensim/models/ldamulticore.html